In [1]:
import pandas as pd
import re
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
df_movies = pd.read_csv("regressionPreprocessing.csv")
print("Length after import: " + str(len(df_movies)))
df_movies = df_movies.fillna(0)
df_movies.head(5)

Length after import: 7517


,budget,id,runtime,vote_count,History,Western,Music,Family,Comedy,Drama,...,orig_ml,orig_es,orig_bn,orig_ab,orig_wo,orig_ca,orig_ru,orig_id,orig_is,director
0,30000000,862,81.0,5415,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1709
1,65000000,8844,104.0,2413,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1645
2,60000000,949,170.0,1886,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2338
3,58000000,710,130.0,1194,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2231
4,98000000,1408,119.0,137,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2812


In [3]:
features_to_remove = ['+18','id','actors','spokenLanguages','productionCountries','productionCompanies','original_language','hasHomepage','part_of_collection']
for i in features_to_remove:
    if i in df_movies.columns:
        df_movies = df_movies.drop(columns=i)
df_movies.head(5)

,budget,runtime,vote_count,History,Western,Music,Family,Comedy,Drama,Foreign,...,orig_ml,orig_es,orig_bn,orig_ab,orig_wo,orig_ca,orig_ru,orig_id,orig_is,director
0,30000000,81.0,5415,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1709
1,65000000,104.0,2413,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1645
2,60000000,170.0,1886,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2338
3,58000000,130.0,1194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2231
4,98000000,119.0,137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2812


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

#features = features.dropna()

# separate features and target variable
rating = df_movies['rating'] # weight
features = df_movies.drop(columns=['rating'])

# create a train/test split
features_train, features_test, rating_train, rating_test = train_test_split(
    features, rating, test_size=0.4, random_state=42)

print("Train: " + str(len(features_train)) + " Features and " + str(len(rating_train)) + " Ratings")
print("Test: " + str(len(features_test)) + " Features and " + str(len(rating_test)) + " Ratings")

Train: 4510 Features and 4510 Ratings
Test: 3007 Features and 3007 Ratings


In [5]:
import matplotlib.pyplot as plt

X = features
y = rating

# Splitting the dataset into the Training set and Test set
# I have a fairly large dataset of +- 4000 entries, so I'm going with 10% test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

#This regressor was picked with gridsearch over many parameters - took 4 hours
from xgboost import XGBRegressor
regressor = XGBRegressor(colsample_bytree= 0.6, gamma= 0.7, max_depth= 4, min_child_weight= 5,
                         subsample = 0.8, objective='reg:squarederror')
regressor.fit(X, y)
y_pred = regressor.predict(X_test)

from sklearn.metrics import r2_score
score = r2_score(y_test, y_pred)
print("Score: ", score)

fig, ax = plt.subplots()
ax.scatter(y_test, y_pred)
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
ax.set_xlabel('Measured rating')
ax.set_ylabel('Predicted rating')
plt.title('Measured versus predicted rating')
plt.ylim((1, 5))   # set the ylim to bottom, top
plt.xlim(1, 5)     # set the ylim to bottom, top
plt.show()

Score:  0.09721710311334641


<Figure size 640x480 with 1 Axes>

In [6]:
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np 

X = features
y = rating

# Splitting the dataset into the Training set and Test set
# I have a fairly large dataset of +- 4000 entries, so I'm going with 10% test data
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest=train_test_split(X, y, test_size=0.15)

xgbr = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:squarederror', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)


xgbr.fit(xtrain, ytrain)

scores = cross_val_score(xgbr, xtrain,ytrain,cv=5)
print("Mean cross-validation score: %.2f" % scores.mean())

kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xgbr, xtrain, ytrain, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

ypred = xgbr.predict(xtest)
mse = mean_squared_error(ytest,ypred)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % np.sqrt(mse))

for index, row in ytest[:10].items():
    print(str(index), str(row))
    print(str(xtest.iloc[[index]]))
    #print("## Value: " + str(row))
    #tmp = xgbr.predict(xtest.iloc[[index]])
    #print("Prediction: " + str(tmp))

x_ax = range(len(ytest))
plt.scatter(x_ax, ytest, s=5, color="blue", label="original")
plt.plot(x_ax, ypred, lw=0.8, color="red", label="predicted")
plt.legend()
plt.show()

Mean cross-validation score: -0.00
K-fold CV average score: -0.01
MSE: 0.38
RMSE: 0.62
1459 3.5330188679245285


IndexError: positional indexers are out-of-bounds